# speed running a dataframe to table

this is a pure dataframe approach to constructing html representations

* the shorter the code distance, the easier it is to reason
* setup your accessors  before solving your problem
    * what libraries will you need to use in dataframe chaining?
    * do they improve your ability to reason?
    * encapsulate the complexity

In [1]:
    from nbconvert_a11y.table import new
    df = DataFrame(numpy.random.randn(100, 100))

In [2]:
    visible = 3
    cranges = [Index(range(visible)), Index(range(len(df.columns)-visible, len(df.columns)))]
    rranges = [Index(range(visible)), Index(range(len(df)-visible, len(df)))]

the table representation establishes the concept of row and columns along with their associated indexes.
our transformation from a dataframe is translating the nominal tidy data columns and indexes
to ordinal table row and column indexes.
we do this this in a few groups 1. the `thead` region, 2. the `tbody` region, 3. the `tfoot` region.

our primary library for building the table is beautiful soup.
spending time building out custom accessors for the dataframe objects 
will prove beneficial when building our final table.

In [3]:
@pandas.api.extensions.register_dataframe_accessor("bs4")
@pandas.api.extensions.register_series_accessor("bs4")
@pandas.api.extensions.register_index_accessor("bs4")
@dataclass
class _BeautifulSoup:
    parent = bs4.BeautifulSoup(features="lxml") 
    object: DataFrame | Series | Index
    cast: typing.Callable = None
    GROUPS = "thead", "tbody", "table", "tr", "tfoot"

    @classmethod
    def new(cls, name, *object, **attrs):
        element = cls.parent.new_tag(name=name, attrs=attrs)
        element.extend(object)
        return element
    
    def _cast(self, object):
        if self.cast is not None: object = self.cast(object)
        if isinstance(object, bs4.Tag): return object
        return str(object)
        
    def element(self, name, **attrs):
        object = self.object
        if isinstance(object, Index): object = object.to_frame()
        if isinstance(object, DataFrame):
            return object.map(self._cast).map(partial(self.new, name, **attrs))
        elif isinstance(object, Series):
            return object.apply(self._cast).apply(partial(self.new, name, **attrs))
        raise TypeError(F"cant create element for type of {type(self.object)}")

    def elements(self, name, axis=1, **attrs):
        object = self.object
        if isinstance(object, Index): object = object.to_frame()
        if isinstance(object, DataFrame):
            return object.map(self._cast).apply(lambda row: self.new(name, *row, **attrs), axis=axis)
        elif isinstance(object, Series):
            return self.new(name, *object.apply(self._cast), **attrs)
        raise TypeError(F"cant create elements for type of {type(self.object)}")
    

    for name in "table tbody thead tfoot tr dl ul ol".split(): locals()[name] = functools.partialmethod(elements, name)
    for name in "th td li dt dd".split(): locals()[name] = functools.partialmethod(element, name)
    del name

In [4]:
%%
after setting up our beautiful soup jigs we can no rip through a dataframe that consolidates to an html table.

1. initialize the thead with the names of the indexes ~~or columns~~

        colgroups = [DataFrame([[*df.index.names]]).bs4.th(scope="col")]
        for crange in cranges:
1. add the column names to the thead

            colgroups.append(DataFrame(df.columns[crange].to_frame().T.values, None, crange).bs4.th(scope="col"))
        
1. initialize the rowgroups with the thead computed

        rowgroups = [pandas.concat(colgroups, axis=1).bs4.tr().bs4.thead()]
        for rrange in rranges:
1. iterate throught the row ranges initializing the column groups with the index names

            colgroups = [DataFrame(df.index[rrange].values, rrange + df.columns.nlevels).bs4.th(scope="row")]
            for crange in cranges:
1. map the dataframe to the oridinal row, group of a table element

                colgroups.append(DataFrame(
                    df.iloc[rrange, crange].values, rrange + df.columns.nlevels, crange + df.index.nlevels
                ).bs4.td())
            rowgroups.append(pandas.concat(colgroups, axis=1).bs4.tr().bs4.tbody())
        
1. aggregate the statistics of the footer
        
        colgroups = []
        index = len(df) + df.columns.nlevels + pandas.RangeIndex(2)
        for crange in cranges:
1. the min and max, the extent, enable us to do lots of cool stuff with plotting

            stats = df.iloc[:, crange].agg("min max".split())
            if not colgroups:
1. write the summary index names

                colgroups.append(DataFrame(stats.index.values, index).bs4.th(scope="row"))
1. write the summary values
                
            colgroups.append(DataFrame(stats.values, index, df.index.nlevels + crange).bs4.td())
1. consolidate the footer

        rowgroups.append(pandas.concat(colgroups, axis=1).bs4.tr().bs4.tfoot())        
1. post process and synthesize the complete table

        table = Series(rowgroups).bs4.table(id="demo")
        table.insert(0, new("caption", "quickest table draw in the pacific northwest"))

{{table}}

None,0,1,2,97,98,99
0,2.2504039209751947,-2.2345675848030893,-0.6104281560127284,0.6020068878327615,1.203908035997032,-1.7136577818868608
1,-1.5662144896915853,-0.4362712027341796,-1.1289212836376208,-0.8857141781786718,0.007626107856577465,-0.4528001992314824
2,-1.6108968695354515,0.6006798243860567,-2.1136251861768622,-0.2679664600579199,1.4807757159023784,1.1963786319147491
97,-0.4205991663877114,0.3876195559265363,0.8932022146952874,-0.965072282031295,-0.32480589021166145,-0.31470531603110863
98,-0.626345645214089,-0.7328430824146266,1.0849752226543594,0.5196105417959229,0.4262418297441081,-0.34981111139307625
99,-0.38399602397773525,0.8058392220376006,-0.15972914177695674,0.12116047467294262,-1.3871299199063132,-1.7465081936039943
min,-1.8154838350026143,-3.396175726528162,-3.902040070737226,-2.6311202211632185,-2.6190849260835383,-2.122234340844054
max,2.6448353616889833,2.38864660860436,2.191688321423365,2.7875339710539526,2.240409048653651,2.0519610479207073
